In [2]:
import pandas as pd
import requests as rq
import json
import re

In [3]:
access_token = 'BQAuJEhKCkaHf5gi7-6OcjtLmHooAkGl7lpRK2yfTuHy1pgsHTSOMNlFI8ByovQpCzmut5A_Zm8RRWv2h7xyacHXX_62gI6ueCuMHLTtNdOP9PSHurHcDl5NhKAd5u-umlxyBhQ1zgl0dflb9Q'
df = pd.read_csv('spotify.csv')

In [4]:
# Get genres of artists from top 200 tracks in U.S. Spotify csv
spotify_genres = {}
for artist in df.drop_duplicates(['Artist'])['Artist']:
    url = 'https://api.spotify.com/v1/search?q=' + artist + '&type=artist'
    headers = {'Accept':'application/json',
               'Content-Type':'application/json',
               'Authorization':'Bearer ' + access_token}
    search = rq.get(url, headers=headers)
    spotify_genres[artist] = json.loads(search.text)['artists']['items'][0]['genres']
spotify_genres

{'2 Chainz': ['dwn trap', 'pop rap', 'rap', 'southern hip hop', 'trap music'],
 '21 Savage': ['dwn trap', 'rap', 'trap music'],
 'A Boogie Wit da Hoodie': ['dwn trap',
  'rap',
  'southern hip hop',
  'trap music'],
 'AJR': ['pop'],
 'Adele': ['dance pop', 'pop', 'post-teen pop'],
 'Alessia Cara': ['dance pop', 'pop', 'post-teen pop'],
 'Aminé': ['dwn trap',
  'pop rap',
  'rap',
  'southern hip hop',
  'trap music',
  'underground hip hop'],
 'Andy Williams': ['adult standards',
  'brill building pop',
  'bubblegum pop',
  'cabaret',
  'christmas',
  'easy listening',
  'lounge',
  'mellow gold',
  'opera',
  'operatic pop',
  'rock-and-roll',
  'soft rock',
  'vocal jazz'],
 'Ariana Grande': ['dance pop', 'pop', 'pop christmas', 'post-teen pop'],
 "Auli'i Cravalho": ['hollywood'],
 'Bebe Rexha': ['dance pop', 'pop', 'post-teen pop', 'tropical house'],
 'Big Sean': ['detroit hip hop', 'pop rap', 'rap', 'trap music'],
 'Bing Crosby': ['adult standards',
  'big band',
  'cabaret',
  'ch

In [13]:
count = {}

for key, value in spotify_genres.items():
    # Get count of all genres
    for genre in value:
        if genre in count:
            count[genre] += 1
        else:
            count[genre] = 1
df2 = pd.DataFrame.from_dict(count, orient='index')
df2.sort_values(by=0, ascending=False)

,0
pop,51
rap,42
pop rap,41
dance pop,34
trap music,30
dwn trap,25
tropical house,24
post-teen pop,21
southern hip hop,19
edm,15


In [8]:
# Get single genre names
genres = {}
# Filter trap
hip_hop = re.compile(r'(hip hop|^rap)',re.MULTILINE)
country = re.compile(r'country')
rock = re.compile(r'rock')

for key, value in spotify_genres.items():
    # Search genre list of each artist
    for genre in value:
        if hip_hop.match(genre):
            genres[key] = 'hip hop'
        else:
            if country.match(genre):
                genres[key] = 'country'
            else:
                if rock.match(genre):
                    genres[key] = 'rock'
                # If not hip hop, country, or rock, discard
df2 = pd.DataFrame.from_dict(genres, orient='index')
df2

,0
Post Malone,hip hop
Logic,hip hop
Cardi B,hip hop
21 Savage,hip hop
Lil Uzi Vert,hip hop
French Montana,hip hop
Kendrick Lamar,hip hop
Gucci Mane,hip hop
Travis Scott,hip hop
A Boogie Wit da Hoodie,hip hop
